# 准备知识

## 网络爬虫（web crawler）

1. **```定义```**：<br>
    - 在互联网中自动采集与整理数据信息<br>
    <br>
2. **```可行性```**：<br>
    - <u>有权限</u>，如果没有权限，便不能爬取，例如银行的后台数据
    - <u>能看到</u>，一些小众论坛只有在登录情况下才能查看，那么需要模拟登录，让电脑“看得到”数据
3. **```伦理```**：<br>
    - 除了一般的研究伦理，如果爬取的是<u>特定地区的论坛</u>，研究报告时可能需要匿名化处理。

## 基本思路

|**步骤**|任务|说明|方法|
|:----:|:----:|:-----|:----:|
|**1**|**生成网址**|分析**<mark>网址规律</mark>**，批量生成网址|for循环，format函数|
|2|请求+获取网页数据|模拟人工打开网页，并将网页存储为数据对象|requests包|
|**3**|**解析数据**|分析数据规律，整理出所需字段|**<mark>pyquery包</mark>**，json包<br>（还有**<mark>lxml包和beautifulsoup4包</mark>**）|
|4|存储数据|使用csv包将数据存储到csv文件中|csv包|
|5|批量爬取|对所有网址循环步骤2-4|for循环|

# 生成网址

## 分析网址规律

1. **```任务对象```**：
    - https://book.douban.com/top250<br>
<br>
<br>
2. **```基本情况```**：
    - 每翻一页，网址中的某个**<u>```数字发生规律性变化```</u>**
    - 每页20本书的信息，最多翻到50页

3. **```规律```**：
- **<u>```原网址```</u>**：
    * https://book.douban.com/top250?start=25
    * https://book.douban.com/top250?start=50

    * ……
    * https://book.douban.com/top250?start=225

<br>
- **<u>```总结规律```</u>**：
    * 第1页，数字为0
    * 第2页，数字为25
    * 第3页，数字为50
    * ……
    * 第10页，数字为225
<br>
<br>
    * <mark><big>第p页，数字为(p-1)*25</big></mark>

## 批量生成网址

In [1]:
# 尝试用format替换数字
'https://book.douban.com/top250?start={num}'.format(num=100)
# 'https://book.douban.com/top250?start={num}'.format(num=0)   # num取0

'https://book.douban.com/top250?start=100'

In [40]:
'https://book.douban.com/top250?start={num}'.format(num=25)  # num取25

'https://book.douban.com/top250?start=25'

In [ ]:
# 简写
template = 'https://book.douban.com/top250?start={num}'
template.format(num=25)

In [2]:
# 循环生成网址，并存入网址列表中
url_list = []    # 生成空列表，用于存储网址
template = 'https://book.douban.com/top250?start={num}'
for p in range(1,11):   # range取1到10
    url = template.format(num=(p-1)*25)    # 取1时，为(1-1)*25，即num为0，以此类推
    url_list.append(url)   # 将上面获得的url【添加进】列表

In [3]:
# 查看url_list情况（获得10页的网址）
url_list

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

In [58]:
# 函数：生成网址  generate_url_list()
# 返回值：url_list为网址列表
def generate_url_list():   # 添加：定义函数名
    url_list = []
    template = 'https://book.douban.com/top250?start={num}'
    for p in range(1,11):
        url = template.format(num=(p-1)*25)
        url_list.append(url)
    return url_list        # 添加：返回网址列表

In [7]:
# 调用函数  generate_url_list()
url_list_tmp = generate_url_list()
url_list_tmp

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

# 请求+获取网页数据	

- 以第1页为例，来请求并获取网页数据
- 例子：https://book.douban.com/top250?start=0

In [8]:
import requests   # 导入requests包

url = 'https://book.douban.com/top250'
resp = requests.get(url)   # 用get向服务器请求获取数据
resp # 查看状态码，返回码418说明访问不成功，需要伪装访问，假装是人工打开网页

<Response [418]>

##  1. **状态码**：


    * 1开头：信息状态码
    * 2开头：成功状态码
    * 3开头：重定向状态码
    * 4开头：客户端错误状态码
    * 5开头：服务端错误状态码

In [9]:
resp.text

''

## 2. **```应对方式```**：

加入请求头headers，一般包括user-agent、cookie和referer等


    * **<u>```user-agent```</u>**：浏览器类型及版本、操作系统及版本、浏览器内核等的信息标识
    * **<u>```cookie```</u>**：用于识别用户身份、记录历史的一段数据（浏览器访问服务器后，服务器传给浏览器的）
    * **<u>```referer```</u>**：告诉服务器该网页是从哪个页面链接过来的

In [10]:
# 尝试加入user-agent
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
resp = requests.get(url, headers=headers)  # 设置参数headers
resp #返回码200说明访问成功

<Response [200]>

In [15]:
# 查看返回内容，搜索该页面的“人类简史”是否在返回的文本中，确认与网页一致
resp.text

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣读书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call

In [11]:
# 将返回内容放入html中，html为str
html = resp.text
html
type(html)

str

In [12]:
# 函数：获得html  get_html(url)
# 参数说明：url为单个网址
# 返回值：html为网址的html数据，即网页源代码的字符串
def get_html(url):         # 添加：定义函数名
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html            # 添加：返回网址的html数据

In [63]:
# 调用函数get_html(url)
url_tmp = 'https://book.douban.com/top250?start=0'  # 第1页
html_tmp = get_html(url_tmp)
html_tmp

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣读书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call

# 解析数据

- 上面获得的html数据是杂乱的，需要把它**整理成结构化的数据**，即解析数据
- 方式：用开发者工具中的elements，分析字段的分布情况，找出具有**定位唯一性的标签**

In [64]:
from pyquery import PyQuery  #导入pyquery包

doc = PyQuery(html) # 将html字符串转换为pyquery数据，便于解析
type(doc)   # 数据类型为pyquery.pyquery.PyQuery
doc.text()    # 数据变“干净”了

'豆瓣读书 Top 250\n!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call(e,o)}catch(e){t(o)}},a=/<script.*?src\\=["\']?([^"\'\\s>]+)/gi,g=/http:\\/\\/(.+?)\\.([^\\/]+).+/i;e.writeln=e.write=function(e){var t,l=a.exec(e);return l&&(t=g.exec(l[1]))?c[t[2]]?void r(e):void(

In [65]:
# 只要书籍信息：调用items获得生成器
doc.items('tr.item')    # 书的定位标签为tr.item

<generator object PyQuery.items at 0x0000029168A23E60>

In [66]:
for book in doc.items('tr.item'):
#     print(type(book))   # 有25本
    print(book)       # 查看这25本书的信息

<tr class="item">
          <td width="100" valign="top">
            <a class="nbg" href="https://book.douban.com/subject/1033825/" onclick="moreurl(this,{i:'0'})">
              <img src="https://img2.doubanio.com/view/subject/s/public/s1024472.jpg" width="90"/>
            </a>
          </td>
          <td valign="top">
            
            <div class="pl2">


              <a href="https://book.douban.com/subject/1033825/" onclick="&quot;moreurl(this,{i:'0'})&quot;" title="顾城的诗">
                顾城的诗

                
              </a>




              
            </div>

              <p class="pl">顾城 / 人民文学出版社 / 1998-12 / 20.00</p>

            

              
              <div class="star clearfix">
                  <span class="allstar45"/>
                  <span class="rating_nums">8.8</span>

                <span class="pl">(
                    26009人评价
                )</span>
              </div>

            
              <p class="quote" style="margin: 10px

**```三种解析情况```**：
1. **<u>```根据定位标签，直接解析获得```</u>**，例如用book('.pl2 a').text()获得“红楼梦”
2. 用第一种方法获得数据，再进行**<u>```list元素提取```</u>**，例如获得['[清] 曹雪芹 著', ' 人民文学出版社 ', ' 1996-12 ', ' 68.00元']，分别提取作者、出版社、出版时间、价格等4个字段
3. 用第一种方法获得数据，再用**<u>```正则表达式提取```</u>**，例如获得“(158378人评价)”，提取“158378（数字）”作为“评价人数（字段）”

<center><font size=4><b>情况1：直接解析</b></font></center>

In [68]:
for book in doc.items('tr.item'):
#     print(book('.info h2 a'))
#     print(book('.pl2 a').attr('title'))   #取书名的两种方式：一是通过attr方法
#     print(book('.pl2 a').text())          # 二是通过text方法，相对完整
    book_name = book('.pl2 a').text()     # 放入book_name字段中
    print(book_name)

顾城的诗
会饮篇
众病之王 : 癌症传
火星救援
父权制与资本主义
上学记
总统是靠不住的 : 近距离看美国之二
李光耀观天下
毛姆短篇小说精选集
目送
规训与惩罚 : 监狱的诞生
亲爱的安德烈
时间的秩序
金锁记
从一到无穷大 : 科学中的事实和臆测
认识电影 : 插图第11版
分成两半的子爵
帷幕 : 阿加莎·克里斯蒂侦探作品集33
跨越边界的社区（修订版） : 北京“浙江村”的生活史
坟
中国哲学简史
青铜时代 : 时代三部曲
荆棘鸟
黑箱：日本之耻
如父如子


In [69]:
for book in doc.items('tr.item'):
    print(book('.pl2 a').text())          

顾城的诗
会饮篇
众病之王 : 癌症传
火星救援
父权制与资本主义
上学记
总统是靠不住的 : 近距离看美国之二
李光耀观天下
毛姆短篇小说精选集
目送
规训与惩罚 : 监狱的诞生
亲爱的安德烈
时间的秩序
金锁记
从一到无穷大 : 科学中的事实和臆测
认识电影 : 插图第11版
分成两半的子爵
帷幕 : 阿加莎·克里斯蒂侦探作品集33
跨越边界的社区（修订版） : 北京“浙江村”的生活史
坟
中国哲学简史
青铜时代 : 时代三部曲
荆棘鸟
黑箱：日本之耻
如父如子


In [70]:
# 用类似方法的字段有：book_name, desc, score
for book in doc.items('tr.item'):
    book_name = book('.pl2 a').text()
    desc = book('span.inq').text()
    score = book('span.rating_nums').text()
    print(book_name, desc, score)

顾城的诗 火焰是我们诗歌唯一的读者 8.8
会饮篇  9.3
众病之王 : 癌症传 癌症传 9.0
火星救援 跟火星来一场不是你死就是我活的过家家游戏 8.9
父权制与资本主义  9.2
上学记 20世纪中国知识分子的心灵史 8.9
总统是靠不住的 : 近距离看美国之二 美国政治法律制度的基本原理和操作细节 8.8
李光耀观天下  9.2
毛姆短篇小说精选集 在各种光怪陆离的场景中，迷失的人性引发了一连串的悲剧 9.1
目送 不必追 8.6
规训与惩罚 : 监狱的诞生 伴随着惩罚的人性化，自由反而日益丧失了 9.3
亲爱的安德烈 弭平代沟，跨越文化阻隔 8.6
时间的秩序  8.9
金锁记 一个小商人家庭出身的女子曹七巧的心灵变迁历程 8.6
从一到无穷大 : 科学中的事实和臆测 科学中的事实和臆测 9.1
认识电影 : 插图第11版 电影入门经典之作 8.9
分成两半的子爵  8.8
帷幕 : 阿加莎·克里斯蒂侦探作品集33  9.1
跨越边界的社区（修订版） : 北京“浙江村”的生活史  9.5
坟  9.6
中国哲学简史 中国哲学入门书 8.9
青铜时代 : 时代三部曲 唐人传奇贯注现代情趣 8.8
荆棘鸟 澳洲乱世情 8.6
黑箱：日本之耻  8.9
如父如子  8.9


<center><font size=4><b>情况2：提取列表元素</b></font></center>

In [71]:
for book in doc.items('tr.item'):
#     info_list = book('.info .pub').text()    # 用左下划线'/'分隔的字符串
    info_list = book('p.pl').text().split('/')     # 以'/'为分隔符split，得到info_list列表
#     print(info_list)
#     print(info_list[-3])    # 用[]提取列表元素，例如倒数第3个是出版社，索引为-3
    publisher = info_list[-3].strip()   # 以同样的方式获得authors, publisher, pub_time, price字段
#     print(publisher)
    pub_time = info_list[-2]
    price = info_list[-1]
    authors = ','.join(info_list[:-3])   # 将多个作者元素，组合到一个字符串里
#     print(authors)
    print(authors, publisher, pub_time, price)

顾城  人民文学出版社  1998-12   20.00
[古希腊] 柏拉图 , 王太庆  商务印书馆  2013-1   12.00元
[美] 悉达多·穆克吉 , 李虎  中信出版社  2013-2   42.00元
[美]安迪•威尔 , 陈灼  译林出版社  2015-10   38.00
[日] 上野千鹤子 , 邹韵  浙江大学出版社  2020-3   58.00元
何兆武 口述  生活·读书·新知三联书店  2006-8   19.80元
林达  生活·读书·新知三联书店  2004-08   21.80
李光耀  北京大学出版社  2015-6   58.00元
[英] 威廉·萨默塞特·毛姆 , 冯亦代  译林出版社  2012-11   36.00元
龙应台  生活·读书·新知三联书店  2009-10   39.00元
[法] 米歇尔·福柯 , 刘北成  生活·读书·新知三联书店  2003-1   21.80元
龙应台  人民文学出版社  2008-12   26.00
[意] 卡洛·罗韦利 , 杨光  湖南科学技术出版社  2019-6   56.00元
张爱玲  哈尔滨出版社  2005-6   13.5元
[美] G. 伽莫夫 , 暴永宁 译  科学出版社  2002-11   29.00元
[美] 路易斯·贾内梯 , 焦雄屏  世界图书出版公司  2007-11   68.00元
[意] 伊塔洛·卡尔维诺 , 吴正仪  译林出版社  2012-4-1   20.00元
[英] 阿加莎·克里斯蒂 , 李杨  新星出版社  2015-2   26.00
项飙  生活·读书·新知三联书店  2018-3   68.00元
鲁迅  人民文学出版社  2006-12   16.00元
冯友兰 , 赵复三  新世界出版社  2004-1   38.00元
王小波  花城出版社  1997-5   29.00元
[澳] 考琳·麦卡洛 , 曾胡  译林出版社  1998-7   28.00元
[日]伊藤诗织 , 匡匡 , 雅众文化 中信出版集团  2019-4   48.00元
[日] 是枝裕和 , 丹勇  湖南文艺出版社  2018-4   49.80


<center><font size=4><b>情况3：正则表达式提取</b></font></center>

In [50]:
import re    # 导入re包

for book in doc.items('tr.item'):
    people_num_raw = book('span.pl').text()
#     print(people_num_raw)
#     print(re.findall('[0-9]+', people_num_raw))    # 匹配people_num_raw字段中的数字，获得列表
#     print(re.findall('[0-9]+', people_num_raw)[0])   # 取列表中的第1个元素，索引为0
    people_num = re.findall('[0-9]+', people_num_raw)[0]  # 放入people_num字段中
    print(people_num)

413884
806547
275814
178124
99448
414700
208438
164241
366759
154292
130421
458438
743056
167773
129084
130360
155483
87150
141849
65888
166331
76813
87335
147835
44970


1. 将这些**```字段组合成字典，放入列表```**中
2. 书名、作者、出版社等作为**```key```**，人类简史、尤瓦尔·赫拉利、中信出版社等作为**```value```**
3. **```例子```**如下：

In [33]:
bookinfo_list_tmp = [
    {'book_name': '人类简史：从动物到上帝', 'author': '尤瓦尔·赫拉利', 'publisher': '中信出版社'},
    {'book_name': '理想国', 'author': '柏拉图', 'publisher': '商务印书馆'},
    {'book_name': '社会契约论', 'author': '卢梭', 'publisher': '商务印书馆'}
    ]
print(bookinfo_list_tmp)
print('数据类型为：',type(bookinfo_list_tmp))   # 类型为list
print('数据长度为：',len(bookinfo_list_tmp))    # 长度为3，即3本书的信息

[{'book_name': '人类简史：从动物到上帝', 'author': '尤瓦尔·赫拉利', 'publisher': '中信出版社'}, {'book_name': '理想国', 'author': '柏拉图', 'publisher': '商务印书馆'}, {'book_name': '社会契约论', 'author': '卢梭', 'publisher': '商务印书馆'}]
数据类型为： <class 'list'>
数据长度为： 3


In [51]:
bookinfo_list = []   # 生成空列表，用于存储书籍信息
doc = PyQuery(html)
for book in doc.items('tr.item'):
    book_name = book('.pl2 a').text()    # 情况1：直接解析
    desc = book('span.inq').text()
    score = book('.rating_nums').text()
    
    info_list = book('p.pl').text().split('/')    # 情况2：提取列表元素
    publisher = info_list[-3].strip()   
    pub_time = info_list[-2]
    price = info_list[-1]
    authors = ''.join(info_list[:-3]) 
    
    people_num_raw = book('span.pl').text()       # 情况3：正则表达式提取
    people_num = re.findall('[0-9]+', people_num_raw)[0]
    
    bookinfo = {'book_name':book_name,   # 为每本书创建一个字典，不同字段建构不同键值对
                 'authors':authors,
                 'publisher':publisher,
                 'pub_time':pub_time,
                 'desc':desc,
                 'score':score,
                 'people_num':people_num,
                 'price':price,
                }
    
    bookinfo_list.append(bookinfo)      # 将字典添加进bookinfo_list列表中

In [52]:
bookinfo_list
# len(bookinfo_list)

[{'book_name': '顾城的诗',
  'authors': '顾城 ',
  'publisher': '人民文学出版社',
  'pub_time': ' 1998-12 ',
  'desc': '火焰是我们诗歌唯一的读者',
  'score': '8.8',
  'people_num': '26009',
  'price': ' 20.00'},
 {'book_name': '会饮篇',
  'authors': '[古希腊] 柏拉图  王太庆 ',
  'publisher': '商务印书馆',
  'pub_time': ' 2013-1 ',
  'desc': '',
  'score': '9.3',
  'people_num': '11008',
  'price': ' 12.00元'},
 {'book_name': '众病之王 : 癌症传',
  'authors': '[美] 悉达多·穆克吉  李虎 ',
  'publisher': '中信出版社',
  'pub_time': ' 2013-2 ',
  'desc': '癌症传',
  'score': '9.0',
  'people_num': '12983',
  'price': ' 42.00元'},
 {'book_name': '火星救援',
  'authors': '[美]安迪•威尔  陈灼 ',
  'publisher': '译林出版社',
  'pub_time': ' 2015-10 ',
  'desc': '跟火星来一场不是你死就是我活的过家家游戏',
  'score': '8.9',
  'people_num': '19457',
  'price': ' 38.00'},
 {'book_name': '父权制与资本主义',
  'authors': '[日] 上野千鹤子  邹韵 ',
  'publisher': '浙江大学出版社',
  'pub_time': ' 2020-3 ',
  'desc': '',
  'score': '9.2',
  'people_num': '16503',
  'price': ' 58.00元'},
 {'book_name': '上学记',
  'authors': '何兆武 口

In [53]:
# 函数：解析数据  extract_bookinfo_list(html)
# 参数说明：html为网页源代码的字符串
# 返回值：bookinfo_list为书籍的字典列表
def extract_bookinfo_list(html):         # 添加：定义函数名
    bookinfo_list = []
    doc = PyQuery(html)
    for book in doc.items('tr.item'):
        try:                             # 添加：try语句，避免特殊网页中断整个循环
            book_name = book('.pl2 a').text()    
            desc = book('span.inq').text()
            score = book('.rating_nums').text()
            info_list = book('p.pl').text().split('/')    
            publisher = info_list[-3].strip()   
            pub_time = info_list[-2]
            price = info_list[-1]
            authors = ''.join(info_list[:-3])
            people_num_raw = book('span.pl').text()       
            people_num = re.findall('[0-9]+', people_num_raw)[0]

            bookinfo = {'book_name':book_name,
                         'authors':authors,
                         'publisher':publisher,
                         'pub_time':pub_time,
                         'desc':desc,
                         'score':score,
                         'people_num':people_num,
                         'price':price,
                        }

            bookinfo_list.append(bookinfo)
        except:                        # 添加：except和pass语句，如果碰到bug，那么跳出此次循环、不执行任何操作，进行下一次循环
            pass
        
    return bookinfo_list               # 添加：返回书籍的字典列表

In [37]:
html

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣读书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call

In [57]:
# 调用函数extract_bookinfo_list(html)
bookinfo_list = extract_bookinfo_list(html)
print(bookinfo_list)
print('书籍数量为：', len(bookinfo_list))

[{'book_name': '顾城的诗', 'authors': '顾城 ', 'publisher': '人民文学出版社', 'pub_time': ' 1998-12 ', 'desc': '火焰是我们诗歌唯一的读者', 'score': '8.8', 'people_num': '26009', 'price': ' 20.00'}, {'book_name': '会饮篇', 'authors': '[古希腊] 柏拉图  王太庆 ', 'publisher': '商务印书馆', 'pub_time': ' 2013-1 ', 'desc': '', 'score': '9.3', 'people_num': '11008', 'price': ' 12.00元'}, {'book_name': '众病之王 : 癌症传', 'authors': '[美] 悉达多·穆克吉  李虎 ', 'publisher': '中信出版社', 'pub_time': ' 2013-2 ', 'desc': '癌症传', 'score': '9.0', 'people_num': '12983', 'price': ' 42.00元'}, {'book_name': '火星救援', 'authors': '[美]安迪•威尔  陈灼 ', 'publisher': '译林出版社', 'pub_time': ' 2015-10 ', 'desc': '跟火星来一场不是你死就是我活的过家家游戏', 'score': '8.9', 'people_num': '19457', 'price': ' 38.00'}, {'book_name': '父权制与资本主义', 'authors': '[日] 上野千鹤子  邹韵 ', 'publisher': '浙江大学出版社', 'pub_time': ' 2020-3 ', 'desc': '', 'score': '9.2', 'people_num': '16503', 'price': ' 58.00元'}, {'book_name': '上学记', 'authors': '何兆武 口述 ', 'publisher': '生活·读书·新知三联书店', 'pub_time': ' 2006-8 ', 'desc': '20世纪中国知识分子

# 存储数据

In [39]:
import csv     # 导入csv包

# 打开文件
file = open('./data/books_250.csv', 'a+', encoding='utf-8', newline='') # 文件名为books_philosophy.csv，模式为a+，如果已有文件，在末尾追加，如果没，则生成新文件；编码为utf-8，需要区分换行符
fieldnames = ['book_name', 'authors', 'publisher', 'pub_time', 'desc', 'score', 'people_num', 'price']  # 设置标题行fieldnames
writer = csv.DictWriter(file, fieldnames=fieldnames)    # 要求以字典的形式写入数据，fieldnames注明字典中键的名称，也就是书名、作者这些标题名
writer.writeheader()  # 将fieldnames设置的标题key写入首行

# 循环写入字典列表：因为有很多本书，需要一行行写入
for bookinfo in bookinfo_list:
    writer.writerow(bookinfo)  # 写入一行书籍信息
    
file.close() # 关闭文件

# 批量爬取

刚才我们做了几项工作：
1. 生成网址————发现网址规律，获得所有网址url_list
2. 请求+获取网页数据————获得html网页源代码字符串（**单个网址**）
3. 解析数据————从html中获得书籍的各字段信息，存储为字典列表（**单个网址**）
4. 存储数据（**单个网址**）
<br>
<br>
<br>
<b><mark>下面对所有网址url_list循环步骤2-4</mark><b>

|**步骤**|任务|函数|输入参数|返回值|
|:----:|:-----|:-----|:-----|:-----|
|1|生成网址|generate_url_list()|--|url_list网址列表|
|**<mark>2</mark>**|**<mark>请求+获取网页数据</mark>**|get_html(url)|单个网址|html网页源代码的字符串|
|**<mark>3</mark>**|**<mark>解析数据</mark>**|extract_bookinfo_list(html)|（单个网址）html网页源代码的字符串|bookinfo_list书籍的字典列表|
|**<mark>4</mark>**|**<mark>存储数据</mark>**|--|--|--|
|5|批量爬取|--|--|--|

In [44]:
url_list = []
template = 'https://book.douban.com/top250?start={num}'
for p in range(1,11):   # range取1到11
    url = template.format(num=(p-1)*25)
    url_list.append(url)
url_list

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

In [46]:
# 打开文件
file = open('./data/books_250.csv', 'a+', encoding='utf-8', newline='') 
fieldnames = ['book_name', 'authors', 'publisher', 'pub_time', 'desc', 'score', 'people_num', 'price'] 
writer = csv.DictWriter(file, fieldnames=fieldnames)    
writer.writeheader() 

# 对所有网址url_list循环步骤2-4
for url in url_list:
    
    html = get_html(url)       # 【步骤2：得到html数据，用函数get_html(url)】
    
    bookinfo_list = extract_bookinfo_list(html)  # 【步骤3：整理成结构化数据，用函数extract_bookinfo_list(html)】
    
    for bookinfo in bookinfo_list:      # 【步骤4：写数据到csv文件】
        writer.writerow(bookinfo)
    
file.close() # 关闭文件

In [55]:
# 函数：爬虫主函数  main(filename)
# 参数说明：filename为文件名称
# 仅执行命令，不返回任何值
def main(filename):         # 添加：定义函数名
    print('开始采集豆瓣读书top 250书籍！')              # 添加：说明“开始采集豆瓣哲学类书籍！”
    
    # 生成所有网址url_list
    url_list = generate_url_list()
    
    # 打开文件
    file = open(filename, 'a+', encoding='utf-8', newline='')     # 修改：将文件名称，设定为函数的参数
    fieldnames = ['book_name', 'authors', 'publisher', 'pub_time', 'desc', 'score', 'people_num', 'price'] 
    writer = csv.DictWriter(file, fieldnames=fieldnames)    
    writer.writeheader() 

    # 对所有网址url_list循环步骤2-4
    for url in url_list:
        print('正在采集：{url}'.format(url=url))    # 添加：说明“正在采集<url>”
        html = get_html(url)
        bookinfo_list = extract_bookinfo_list(html)
        for bookinfo in bookinfo_list:
            writer.writerow(bookinfo)

    file.close()

    print('采集完毕！')            # 添加：说明“采集完毕！”

In [59]:
main(filename='./data/books_250.csv')

开始采集豆瓣读书top 250书籍！
正在采集：https://book.douban.com/top250?start=0
正在采集：https://book.douban.com/top250?start=25
正在采集：https://book.douban.com/top250?start=50
正在采集：https://book.douban.com/top250?start=75
正在采集：https://book.douban.com/top250?start=100
正在采集：https://book.douban.com/top250?start=125
正在采集：https://book.douban.com/top250?start=150
正在采集：https://book.douban.com/top250?start=175
正在采集：https://book.douban.com/top250?start=200
正在采集：https://book.douban.com/top250?start=225
采集完毕！


# 参考：
- [Excel打开监控数据CSV文件乱码如何处理？_云监控服务 CES_常见问题_产品咨询_华为云 ](https://support.huaweicloud.com/ces_faq/ces_faq_0018.html)